#  Assistants With Vectore store

- Executed in colab

# This creates a OpenAI assistant V2

## Connect to openai

In [1]:
from openai import OpenAI
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI()

In [3]:
description = """
  You are a `Wonka Milk Chocolate Factory Bot` for `The Code Cruise`, who responds to the
  employees efficiently with precise response regarding the company
  Wonka Milk Chocolate Factory: Facility Safety Rules.
"""

In [4]:
instructions = """
  Do not engage in any other conversation that isn't related to the policies of `Wonka Milk Chocolate Factory`
  or information regarding the company, in case the user is asking questions outside of it then
  excuse yourself from the conversation by responding as follows:

  'I apologize but as a bot I can only guide you regarding
  the policies and company specific information.'
"""

In [5]:
assistant = client.beta.assistants.create(
    name="The Cruise Assistant",
    description=description,
    instructions=instructions,
    model="gpt-4-turbo",
    tools=[{ "type": "file_search" }],
)

In [15]:
assistant

Assistant(id='asst_7JFJkkFFe2QXyxeJcOUPRthm', created_at=1736568744, description='\n  You are a `Wonka Milk Chocolate Factory Bot` for `The Code Cruise`, who responds to the \n  employees efficiently with precise response regarding the company \n  Wonka Milk Chocolate Factory: Facility Safety Rules.\n', instructions="\n  Do not engage in any other conversation that isn't related to the policies of `Wonka Milk Chocolate Factory` \n  or information regarding the company, in case the user is asking questions outside of it then \n  excuse yourself from the conversation by responding as follows:\n  \n  'I apologize but as a bot I can only guide you regarding \n  the policies and company specific information.'\n", metadata={}, model='gpt-4-turbo', name='The Cruise Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='

In [16]:
assistant_id = assistant.id
assistant_id

'asst_7JFJkkFFe2QXyxeJcOUPRthm'

# This helps in running a OpenAI Assistant V2 by connect to vector store


## Create Vector Store

In [6]:
# Create Vector Store
vector_store = client.beta.vector_stores.create(name="company_bot")
print(f"Vector Store Id - {vector_store.id}")

Vector Store Id - vs_iISpzkIJEdvuAppuR41dzQXz


In [7]:
vector_store

VectorStore(id='vs_iISpzkIJEdvuAppuR41dzQXz', created_at=1736568795, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1736568795, metadata={}, name='company_bot', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)

## Upload Wonka Milk Chocolate Factory File
 Uload this file in colab files folder - Wonka Chocolate Facility Rules.pdf

In [8]:
# Upload Work From Home Policy File
file_paths = ["Wonka Chocolate Facility Rules.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

In [9]:
file_streams

[<_io.BufferedReader name='Wonka Chocolate Facility Rules.pdf'>]

## Add Files To Vector Store in batch - file_batches

In [10]:
 # Add Files To Vector Store
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id = vector_store.id,
  files = file_streams
)

In [12]:
file_batch

VectorStoreFileBatch(id='vsfb_d29901dfc2ca417e9887de56fbed96ce', created_at=1736568891, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), object='vector_store.file_batch', status='completed', vector_store_id='vs_iISpzkIJEdvuAppuR41dzQXz')

## Check The Status Of Files

In [11]:
# Check The Status Of Files
print(f"File Status {file_batch.status}")

File Status completed


## Update The Assistant With A Vector Store

In [17]:
assistant_id

'asst_7JFJkkFFe2QXyxeJcOUPRthm'

In [18]:
# Update The Assistant With A Vector Store
assistant = client.beta.assistants.update(
  assistant_id=assistant_id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
print("Assistant Updated with vector store!")

Assistant Updated with vector store!


## Create A Thread

In [19]:
# Create A Thread
thread = client.beta.threads.create()
print(f"Your thread id is - {thread.id}\n\n")

Your thread id is - thread_6E3lsuX2kJ7OQWcXRxSksWHK




## Run a loop where user can ask questions

In [21]:
# Run a loop where user can ask questions
while True:
    text = input("What's your question?\n") # Its front UI Q

    if text=="exit": # this is to break loop
      break

    #Add 1st message/text to thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=text,
    )

    #Create run by adding thread id and assistant id
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    #In above step it creates run and executes also, now check the message from list, which will have response
    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    message_content = messages[0].content[0].text
    print("Response: \n")
    print(f"{message_content.value}\n")


What's your question?
what step follwed in Ingredients Handling:
Response: 

Here are the steps followed in Ingredient Handling at the Wonka Milk Chocolate Factory:

1. Personnel are required to wear gloves and respiratory protection to manage dusty or powdered ingredients.
2. Employees must adhere to precise measuring and weighing methods to ensure consistency and quality in production.
3. There is a heightened attention to allergen control and the potential for cross-contamination, mandating specific preventive practices【6:0†source】.

What's your question?
what steps for Mixing and Grinding:
Response: 

The steps followed in the Mixing and Grinding process at the Wonka Milk Chocolate Factory are:

1. Workers need to maintain a safe distance from mixers and grinders while these machines are operating to ensure safety.
2. Only designated tools and utensils should be used for handling molten chocolate.
3. Employees must be vigilant about the risks of burns and spills and follow establis